In [1]:
import pandas as pd

In [2]:
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [13]:
pd.DataFrame([i for i in range(10)]).quantile(0.25, axis=0)

0    2.25
Name: 0.25, dtype: float64

In [148]:
symbol = 'btc'

In [149]:
data = pd.read_csv('data/pred/pred_stw_brt_si1_' + symbol + '.csv', index_col=0, parse_dates=True)[1:]
prices = pd.read_csv('data/prices_open/prices_open_' + symbol + '.csv', index_col=0, parse_dates=True)[1:]

In [150]:
# data = data - data.min()
# data = data / data.max()
# data = data * 2 - 1

In [151]:
(prices.price_open[-1] - prices.price_open[0]) / prices.price_open[0]

0.46040904944628314

In [152]:
th = 0.1
dec_history = []
capital = 100000
assets = 0
i = 0
for s, p in zip(data.pred[:-1], prices.price_open):
    if i == 0:
        i += 1
        continue
    curr_min = data[:i].min().values
    curr_max = data[:i].max().values
#     s = (s - curr_min) / curr_max
#     s = s * 2 - 1
    decision = 0
    if s > data[:i].quantile(0.6).values[0]:
        decision = 1
    if s < data[:i].quantile(0.4).values[0]:
        decision = -1
    
    if decision == 1 and capital > 0:
        assets = capital / p
        capital = 0
    elif decision == -1 and assets > 0:
        capital = assets * p
        assets = 0
    dec_history.append([s, decision, capital, p, assets, (max(capital, assets * p) - 100000) / 100000, (p - prices.price_open[0]) / prices.price_open[0]])
    i += 1
final = max(capital, assets * prices.price_open[-1]) 
print(final, (final - 100000) / 100000)

180354.6095477885 0.8035460954778851


In [153]:
pd.DataFrame(dec_history, columns=['sent', 'decision', 'capital', 'price', 'assets', 'o', 'm']).replace({'decision' : {1 : 'buy', -1 : 'sell'}}).to_csv('tmp_dec_history.csv')

In [154]:
a = pd.DataFrame(dec_history, columns=['sent', 'decision', 'capital', 'price', 'assets', 'o', 'm'])

In [155]:
R = a['o']

In [156]:
r = R.diff()

In [166]:
sr = r.mean() /r.std() * np.sqrt(272)

In [169]:
r.mean()

0.0020498624884639923

In [167]:
import numpy as np

In [168]:
sr

-26.933424352631004